In [1]:
import os
from tvb.simulator.lab import *
import numpy as np
import seaborn
import matplotlib.pyplot as plt
import pickle as cPickle
from tvb.simulator.models.stefanescu_jirsa import ReducedSetHindmarshRose
from os.path import join as pjoin
import pandas as pd
import scipy.io

/Users/yat-lok/miniconda3/envs/tvbenv/lib/python3.10/site-packages/tvb/datatypes/surfaces.py:60: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [2]:
def tvb_K21_fitting(k21, Go, file):
    connectivity.speed = np.array([10.])
    sim = simulator.Simulator(
        model=ReducedSetHindmarshRose(K21=np.array([k21])), 
        connectivity=connectivity.Connectivity.from_file(file),                      
        coupling=coupling.Linear(a=np.array([Go])),
        simulation_length=1e3*416,
        integrator=integrators.HeunStochastic(dt=0.01220703125, noise=noise.Additive(nsig=np.array([1.0]), ntau=0.0,
                                                                                    random_stream=np.random.RandomState(seed=42))),
        monitors=(
        monitors.TemporalAverage(period=1.),
        monitors.Bold(hrf_kernel = equations.Gamma(), period=2000.),
        monitors.ProgressLogger(period=1e2)
        ))
    sim.configure()
    (tavg_time, tavg_data), (raw_time, raw_data),_ = sim.run()
    return raw_data

In [3]:
K21_range = np.arange(0.0, 3.0, 0.1)
K21_range = [np.round(k21,1) for k21 in K21_range]

corr_df = pd.DataFrame(columns = ['caseid', 'group', 'Go', *K21_range])
print(corr_df.columns)

Index(['caseid',  'group',     'Go',      0.0,      0.1,      0.2,      0.3,
            0.4,      0.5,      0.6,      0.7,      0.8,      0.9,      1.0,
            1.1,      1.2,      1.3,      1.4,      1.5,      1.6,      1.7,
            1.8,      1.9,      2.0,      2.1,      2.2,      2.3,      2.4,
            2.5,      2.6,      2.7,      2.8,      2.9],
      dtype='object')


In [4]:

data_dir = '/Users/yat-lok/workspaces/data4project/'
connectome_dir = pjoin(data_dir, 'lateralization/connectome/zip')
ts_dir = pjoin(data_dir, 'lateralization/ts_fmri/fmri_AAL_16/')

# read mega_table
mega_table = pd.read_excel(pjoin(data_dir, 'mega_table.xlsx'), sheet_name='hodgepodge', skiprows=1)

K21_range = np.arange(0.0, 3.0, 0.1)
_K21_range = [np.round(k21,1) for k21 in K21_range]
# create a dataframe to store the correlation by each K21
corr_df = pd.DataFrame(columns = ['caseid', 'group', 'Go', *_K21_range])
for case, grp, go in zip(mega_table['caseid'], mega_table['group'], mega_table['Go']):
    K21_corr = []
    
    for K21 in K21_range:
        K21 = np.round(K21, 1)
        file = pjoin(connectome_dir, grp, case +'.zip')


        raw_data = tvb_K21_fitting(K21, go, file)
        raw_data_mean = np.mean(raw_data, axis=3)
        # calculate correlation of the matrix
        s_corr = raw_data_mean[:,0,:].corr()

        # read mat file
        mat = scipy.io.loadmat(pjoin(ts_dir, grp, case,'.mat'))
        roi_signal = mat['ROISignal']
        df_roi = pd.DataFrame(roi_signal, columns = ['aCNG-L', 'aCNG-R','mCNG-L','mCNG-R','pCNG-L','pCNG-R', 'HIP-L','HIP-R','PHG-L','PHG-R','AMY-L','AMY-R', 'sTEMp-L','sTEMP-R','mTEMp-L','mTEMp-R'])
        e_corr = df_roi.corr()

        # take the top triangle of the matrix using triu
        e_corr = np.triu(e_corr, k=1)
        s_corr = np.triu(s_corr, k=1)

        # calculate the correlation of the two matrices
        s_corr = s_corr.flatten()
        e_corr = e_corr.flatten()
        # pearson correlation of two flattened matrices
        corr_pearson = np.corrcoef(s_corr, e_corr)[0,1]
        K21_corr.append(corr_pearson)
    corr_df.loc[len(corr_df)] = [case, grp, go, *K21_corr]

    plt.plot(K21_range, K21_corr)
    plt.show()


WARNING  File 'average_orientations' not found in ZIP.
WARNING  File 'cortical' not found in ZIP.
WARNING  File 'hemispheres' not found in ZIP.
WARNING  File 'areas' not found in ZIP.


KeyError: 'factorial'